In [0]:
!pip install smote-variants

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Loading the necessary library files
import pandas as pd

In [4]:
# Loading data from the drive

# Please change the filename as per the location where the file is stored
filename = '/content/drive/My Drive/Packt_Colab/bank-full.csv'
# Loading the data using pandas

bankData = pd.read_csv(filename,sep=";")
bankData.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


**Feature engineering steps**

Let us now do some feature engineering to the data. First we will scale the numerical data and then convert the ordinal data to 
dummy data

In [5]:
# Normalising data
from sklearn import preprocessing

minmaxScaler = preprocessing.MinMaxScaler()

# Converting each of the columns to scaled version

bankData['ageScaled'] = minmaxScaler.fit_transform(bankData['age'].values.reshape(-1,1))
bankData['balScaled'] = minmaxScaler.fit_transform(bankData['balance'].values.reshape(-1,1))
bankData['durScaled'] = minmaxScaler.fit_transform(bankData['duration'].values.reshape(-1,1))

# Dropping the original columns

bankData.drop(['age','balance','duration'], axis=1, inplace=True)

# Print the head of the data

bankData.head()

,job,marital,education,default,housing,loan,contact,day,month,campaign,pdays,previous,poutcome,y,ageScaled,balScaled,durScaled
0,management,married,tertiary,no,yes,no,unknown,5,may,1,-1,0,unknown,no,0.519481,0.092259,0.053070
1,technician,single,secondary,no,yes,no,unknown,5,may,1,-1,0,unknown,no,0.337662,0.073067,0.030704
2,entrepreneur,married,secondary,no,yes,yes,unknown,5,may,1,-1,0,unknown,no,0.194805,0.072822,0.015453
3,blue-collar,married,unknown,no,yes,no,unknown,5,may,1,-1,0,unknown,no,0.376623,0.086476,0.018707
4,unknown,single,unknown,no,no,no,unknown,5,may,1,-1,0,unknown,no,0.194805,0.072812,0.040260


In [0]:
# Converting all the categorical variables to dummy variables
bankCat = pd.get_dummies(bankData[['job','marital','education','default','housing','loan','contact','month','poutcome']])

In [7]:
# Seperating the numerical data
bankNum = bankData[['ageScaled','balScaled','day','durScaled','campaign','pdays','previous']]
bankNum.shape

(45211, 7)

In [8]:
# Merging with the original data frame
# Preparing the X variables
X = pd.concat([bankCat, bankNum], axis=1)
print(X.shape)
# Preparing the Y variable
Y = bankData['y']
print(Y.shape)
X.head()

(45211, 51)
(45211,)


,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,ageScaled,balScaled,day,durScaled,campaign,pdays,previous
0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.519481,0.092259,5,0.053070,1,-1,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.337662,0.073067,5,0.030704,1,-1,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.194805,0.072822,5,0.015453,1,-1,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.376623,0.086476,5,0.018707,1,-1,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.194805,0.072812,5,0.040260,1,-1,0


In [0]:
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

**Undersampling Method.**






In [10]:
# let us first join the train_x and train_y for ease of operation

trainData = pd.concat([X_train,y_train],axis=1)
trainData.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,ageScaled,balScaled,day,durScaled,campaign,pdays,previous,y
19100,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.428571,0.074864,5,0.046970,1,-1,0,no
37958,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0.415584,0.073929,14,0.074624,2,289,19,no
12451,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.272727,0.081619,1,0.095771,3,-1,0,no
18263,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.532468,0.072803,31,0.010573,8,-1,0,no
5128,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.181818,0.075118,21,0.017487,2,-1,0,no


In [11]:
# Finding the indexes of the sample data set where the propensity is 'yes'
ind = trainData[trainData['y']=='yes'].index
print(len(ind))

# Seperate the minority classes
minData = trainData.loc[ind]
print(minData.shape)

# Finding indexes of majority class
ind1 = trainData[trainData['y']=='no'].index
print(len(ind1))
# Seperating the majority class
majData = trainData.loc[ind1]
print(majData.shape)
majData.head()

3723
(3723, 52)
27924
(27924, 52)


,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,ageScaled,balScaled,day,durScaled,campaign,pdays,previous,y
19100,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.428571,0.074864,5,0.046970,1,-1,0,no
37958,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0.415584,0.073929,14,0.074624,2,289,19,no
12451,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.272727,0.081619,1,0.095771,3,-1,0,no
18263,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.532468,0.072803,31,0.010573,8,-1,0,no
5128,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.181818,0.075118,21,0.017487,2,-1,0,no


In [12]:
# Take a random sample equal to length of the minority class to make the data set balanced

majSample = majData.sample(n=len(ind),random_state = 123)
print(majSample.shape)
majSample.head()



(3723, 52)


,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,ageScaled,balScaled,day,durScaled,campaign,pdays,previous,y
17387,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0.402597,0.086131,28,0.017893,3,-1,0,no
34679,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0.428571,0.077933,5,0.031924,7,250,3,no
26572,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.363636,0.098851,20,0.030704,2,-1,0,no
3280,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.506494,0.101157,15,0.036194,1,-1,0,no
4434,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.246753,0.101629,20,0.034160,1,-1,0,no


In [13]:
# Concatenating both data sets and then shuffling the data set

balData = pd.concat([minData,majSample],axis = 0)
print('balanced data set shape',balData.shape)

# Shuffling the data set

from sklearn.utils import shuffle

balData = shuffle(balData)
balData.head()

balanced data set shape (7446, 52)


,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,education_unknown,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,ageScaled,balScaled,day,durScaled,campaign,pdays,previous,y
29631,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0.168831,0.071714,3,0.020333,3,201,2,no
31150,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.571429,0.091152,26,0.073404,1,-1,0,yes
43772,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0.038961,0.074755,24,0.077674,1,89,7,yes
40579,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0.207792,0.074111,21,0.043310,3,89,16,no
36997,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0.389610,0.102464,13,0.001220,5,-1,0,no


In [14]:
# Making the new X_train and y_train

X_trainNew = balData.iloc[:,0:51]
X_trainNew.head()

y_trainNew = balData['y']
y_trainNew.head()

29631     no
31150    yes
43772    yes
40579     no
36997     no
Name: y, dtype: object

In [16]:
from sklearn.linear_model import LogisticRegression
# Defining the LogisticRegression function for Undersampling
bankModel1 = LogisticRegression()
bankModel1.fit(X_trainNew, y_trainNew)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

**SMOTE**



In [18]:
import smote_variants as sv
import numpy as np

# Instantiating the SMOTE class
oversampler= sv.SMOTE()

# Creating new training set

X_train_smote, y_train_smote = oversampler.sample(np.array(X_train), np.array(y_train))


2019-10-10 11:47:17,006:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


In [19]:
# Training the model with Logistic regression model

# Defining the LogisticRegression function

bankModel2 = LogisticRegression()

bankModel2.fit(X_train_smote, y_train_smote)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

**MSMOTE**



In [20]:
import smote_variants as sv
import numpy as np
# Instantiating the MSMOTE class
oversampler= sv.MSMOTE()
# Creating new training sets
X_train_msmote, y_train_msmote = oversampler.sample(np.array(X_train), np.array(y_train))

2019-10-10 11:49:59,250:INFO:MSMOTE: Running sampling via ('MSMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


In [21]:
# Fitting model

# Training the model with Logistic regression model

# Defining the LogisticRegression function
bankModel3 = LogisticRegression()
bankModel3.fit(X_train_msmote, y_train_msmote)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

**Predictions with all three models**

Let us create separate predictions with each model

In [0]:
# Predicting using Under sampler
pred_us = bankModel1.predict(X_test)

# Predicting using SMOTE
pred_smote = bankModel2.predict(X_test)

# Predicting using MSMOTE
pred_msmote = bankModel3.predict(X_test)


In [23]:
# Printing accuracy of each model

print('Accuracy of Logistic regression model prediction on test set for Random Undersampled data set: {:.2f}'.format(bankModel1.score(X_test, y_test)))

print('Accuracy of Logistic regression model prediction on test set for SMOTE data set: {:.2f}'.format(bankModel2.score(X_test, y_test)))

print('Accuracy of Logistic regression model prediction on test set for MSMOTE data set: {:.2f}'.format(bankModel3.score(X_test, y_test)))

Accuracy of Logistic regression model prediction on test set for Random Undersampled data set: 0.85
Accuracy of Logistic regression model prediction on test set for SMOTE data set: 0.85
Accuracy of Logistic regression model prediction on test set for MSMOTE data set: 0.84


In [0]:
# Confusion Matrix & Classification reports for the model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [25]:
# Metrics for Random undersample data set
print(confusion_matrix(y_test, pred_us))

print(classification_report(y_test, pred_us))

[[10250  1748]
 [  311  1255]]
              precision    recall  f1-score   support

          no       0.97      0.85      0.91     11998
         yes       0.42      0.80      0.55      1566

    accuracy                           0.85     13564
   macro avg       0.69      0.83      0.73     13564
weighted avg       0.91      0.85      0.87     13564



In [26]:
# Metrics for SMOTE data set
print(confusion_matrix(y_test, pred_smote))

print(classification_report(y_test, pred_smote))

[[10312  1686]
 [  333  1233]]
              precision    recall  f1-score   support

          no       0.97      0.86      0.91     11998
         yes       0.42      0.79      0.55      1566

    accuracy                           0.85     13564
   macro avg       0.70      0.82      0.73     13564
weighted avg       0.91      0.85      0.87     13564



In [27]:
# Metrics for MSMOTE data set
print(confusion_matrix(y_test, pred_msmote))

print(classification_report(y_test, pred_msmote))

[[10112  1886]
 [  308  1258]]
              precision    recall  f1-score   support

          no       0.97      0.84      0.90     11998
         yes       0.40      0.80      0.53      1566

    accuracy                           0.84     13564
   macro avg       0.69      0.82      0.72     13564
weighted avg       0.90      0.84      0.86     13564

